<a href="https://colab.research.google.com/github/Ansebi/causal_inference/blob/kls/bnlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install bnlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install d3blocks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.5
    Uninstalling MarkupSafe-2.1.5:
      Successfully uninstalled MarkupSafe-2.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
werkzeug 3.0.1 requires MarkupSafe>=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split

from pgmpy.estimators import K2Score, BicScore, BDeuScore, StructureScore

import bnlearn as bn


# Data

In [ ]:
def normalize(
    array: np.array,
    min_: float = None,
    max_: float = None
):
  if len(np.unique(array)) == 1:
    value = np.unique(array)[0]
    norm_value = 0.5
    if max_ != min_:
      norm_value = (value - min_) / (max_ - min_)
    return np.ones_like(array) * norm_value
  if min_ is None:
    min_ = array.min()
  if max_ is None:
    max_ = array.max()
  return (array - min_) / (max_ - min_)

In [ ]:
# DANGER! OVERKILL HAZARD

import random

N_SAMPLES = 1000
N_GARBAGE = 5


source_a = np.random.randint(-100, -10, N_SAMPLES)
source_b = np.random.randint(0, 100, N_SAMPLES)
source_c = np.random.random(N_SAMPLES)


chain_a0 = np.ones(N_SAMPLES)
chain_b0 = np.ones(N_SAMPLES)
chain_c0 = np.ones(N_SAMPLES)

for i in range(N_SAMPLES):
  if source_a[i] >= -30:
    value = 5
  elif source_a[i] <= -70:
    value = 7
  else:
    value = np.sin(source_a[i])
  chain_a0[i] = value

for i in range(N_SAMPLES):
  if source_b[i] < 5:
    value = np.sin(source_b[i]) ** 2
  elif source_b[i] == 5:
    value = 2 * source_b[i]
  else:
    value = 1 / max([source_b[i],  1])
  chain_b0[i] = value

for i in range(N_SAMPLES):
  if source_c[i] < 0.5:
    value = (source_c[i] - 1) * 2
  else:
    value = source_c[i] / (np.cos(source_c[i]) - 1.1)
  chain_c0[i] = value


chain_a1 = np.ones(N_SAMPLES)
chain_b1 = np.ones(N_SAMPLES)
chain_c1 = np.ones(N_SAMPLES)

for i in range(N_SAMPLES):
  value = chain_a0[i]
  if '2' in str(round(value, 3)):
    value = np.sin(value)
  elif sum([int(char) for char in str(np.abs(value)).replace('.', '')]) % 2:
    value = 0.35
  else:
    value = 1 / min([value, 0.1])
  chain_a1[i] = value

for i in range(N_SAMPLES):
  value = chain_b0[i]
  if value > 0.8:
    value = np.tan(value)
  elif np.sin(value) > 0.5:
    value = -0.12
  else:
    value = (1 / min([value, -1])) ** 2
  chain_b1[i] = value

for i in range(N_SAMPLES):
  value = chain_c0[i]
  if value < 0:
    value = np.tan(value)
  else:
    value = 6
  chain_c1[i] = value


outcome = chain_a1 + chain_b1 + chain_c1

garbage = {
    f'garbage_{i}': random.choice(
        [
            np.random.randint(0, 9, N_SAMPLES),
            np.random.randint(0, 100, N_SAMPLES),
            np.random.random(N_SAMPLES)
        ]
    )
    for i in range(N_GARBAGE)
}

df_overkill = pd.DataFrame(
    {
      'source_a': source_a,
      'chain_a0': chain_a0,
      'chain_a1': chain_a1,
      'source_b': source_b,
      'chain_b0': chain_b0,
      'chain_b1': chain_b1,
      'source_c': source_c,
      'chain_c0': chain_c0,
      'chain_c1': chain_c1,
      'outcome': outcome,
      **garbage
    }
)

df = df_overkill.astype(float)

In [ ]:
N_GARBAGE = 3
# magnitude = np.array([1, 7, 9, 2, 1] * 10)
magnitude = np.random.randint(0, 9, 1000)

depends_on_magnitude = magnitude * 10 + np.random.rand(len(magnitude)) / 10
switch = np.ones_like(magnitude)
garbage = {f'garbage_{i}': np.random.random(len(magnitude)) for i in range(N_GARBAGE)}
outcome = depends_on_magnitude + 1 + np.random.rand(len(magnitude)) / 10
df = pd.DataFrame(
    {
        'magnitude': magnitude,
        'depends_on_magnitude': depends_on_magnitude,
        'switch': switch,
        **garbage,
        'outcome': outcome
    }
)

In [ ]:
df_norm = df.apply(normalize)
df_norm.head()

,source_a,chain_a0,chain_a1,source_b,chain_b0,chain_b1,source_c,chain_c0,chain_c1,outcome,garbage_0,garbage_1,garbage_2,garbage_3,garbage_4
0,0.820225,0.749997,0.183136,0.686869,0.001471,0.928321,0.442724,0.910300,0.854709,0.854709,0.787879,0.202020,0.875,0.625,0.507605
1,0.471910,0.000880,0.183136,1.000000,0.001010,0.928321,0.439686,0.905430,0.854689,0.854689,0.191919,0.545455,0.375,0.625,0.320645
2,0.797753,0.749997,0.183136,0.858586,0.001176,0.928321,0.392055,0.829069,0.854305,0.854305,0.838384,0.000000,1.000,0.000,0.613554
3,0.393258,0.021636,0.091225,0.636364,0.001587,0.928321,0.712036,0.137632,0.857063,0.856397,0.040404,0.313131,0.750,1.000,0.603933
4,0.370787,0.231932,1.000000,0.070707,0.014286,0.928321,0.458959,0.936328,0.854804,0.860716,0.575758,0.909091,0.875,0.500,0.988670


# Preprocessing

In [ ]:
# encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

discretized_data = discretizer.fit_transform(df_norm)
discretized_data = pd.DataFrame(discretized_data, columns=df_norm.columns.values)

In [ ]:
discretized_data.head()

,source_a,chain_a0,chain_a1,source_b,chain_b0,chain_b1,source_c,chain_c0,chain_c1,outcome,garbage_0,garbage_1,garbage_2,garbage_3,garbage_4
0,4.0,3.0,1.0,3.0,1.0,1.0,2.0,4.0,1.0,1.0,3.0,0.0,4.0,3.0,2.0
1,2.0,0.0,1.0,4.0,0.0,1.0,2.0,4.0,1.0,1.0,0.0,2.0,2.0,3.0,1.0
2,3.0,3.0,1.0,4.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0,0.0,4.0,0.0,2.0
3,1.0,0.0,0.0,3.0,1.0,1.0,3.0,1.0,2.0,1.0,0.0,1.0,3.0,4.0,2.0
4,1.0,1.0,1.0,0.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,4.0,4.0,2.0,4.0


In [ ]:
discretized_data.shape

(1000, 15)

# Structure learning

In [ ]:
model = bn.structure_learning.fit(discretized_data, methodtype="hc", scoretype="k2")
# model = bn.structure_learning.fit(discretized_data, methodtype="hc", scoretype="bic")
# model = bn.structure_learning.fit(discretized_data, methodtype="hc", scoretype="bdeu", verbose=5)

model = bn.independence_test(model, discretized_data, alpha=0.05, prune=True)

G = bn.plot(model, interactive=True)

[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [k2]
[bnlearn] >Compute structure scores for model comparison (higher is better).
[bnlearn] >Compute edge strength with [chi_square]
[bnlearn] >Edge [source_a <-> chain_c0] [P=0.319181] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [chain_a0 <-> chain_c0] [P=0.694586] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [chain_a1 <-> chain_c0] [P=0.679133] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [source_b <-> chain_c0] [P=0.10235] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [chain_b0 <-> outcome] [P=0.0648053] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [chain_b1 <-> outcome] [P=0.317996] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [chain_b1 <-> chain_c0] [P=0.829292] is exc

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Converting source-target into adjacency matrix..
[d3blocks] >INFO> Making the matrix symmetric..
[d3blocks] >INFO> Set directed=True to see the markers!
[d3blocks] >INFO> Keep only edges with weight>0
[d3blocks] >INFO> Converting source-target into adjacency matrix..
[d3blocks] >INFO> Making the matrix symmetric..
[d3blocks] >INFO> Converting adjacency matrix into source-target..


[bnlearn] >Set edge properties.


[d3blocks] >INFO> Number of unique nodes: 10
[d3blocks] >INFO> Slider range is set to [0, 10]
[d3blocks] >INFO> Write to path: [/tmp/tmp6o_s8b1o/d3graph.html]
[d3blocks] >INFO> File already exists and will be overwritten: [/tmp/tmp6o_s8b1o/d3graph.html]
[d3blocks] >INFO> Keep only edges with weight>0
[d3blocks] >INFO> Converting source-target into adjacency matrix..
[d3blocks] >INFO> Making the matrix symmetric..
[d3blocks] >INFO> Number of unique nodes: 10
[d3blocks] >INFO> Slider range is set to [0, 10]
[d3blocks] >INFO> Write to path: [/tmp/tmpu2stgimk/bnlearn_Directed_Acyclic_Graph_(DAG).html]
[d3blocks] >INFO> File already exists and will be overwritten: [/tmp/tmpu2stgimk/bnlearn_Directed_Acyclic_Graph_(DAG).html]


In [ ]:
# Матрица сопряжённости
print(model['adjmat'])

target                magnitude  depends_on_magnitude  switch  garbage_0  \
source                                                                     
magnitude                 False                  True   False      False   
depends_on_magnitude      False                 False   False      False   
switch                    False                 False   False      False   
garbage_0                 False                 False   False      False   
garbage_1                 False                 False   False      False   
garbage_2                 False                 False   False      False   
outcome                   False                  True   False      False   

target                garbage_1  garbage_2  outcome  
source                                               
magnitude                 False      False     True  
depends_on_magnitude      False      False    False  
switch                    False      False    False  
garbage_0                 False      False   

In [ ]:
from google.colab import files

# Specify the path to the file
file_path = "/tmp/tmpu2stgimk/bnlearn_Directed_Acyclic_Graph_(DAG).html"  # replace "your_file.txt" with your actual file name

# Download the file
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# обучение параметров (весов)
model_params = bn.parameter_learning.fit(model, discretized_data, methodtype='maximumlikelihood')

[bnlearn] >Parameter learning> Computing parameters using [maximumlikelihood]
[bnlearn] >Converting [<class 'pgmpy.base.DAG.DAG'>] to BayesianNetwork model.
[bnlearn] >Converting adjmat to BayesianNetwork.
[bnlearn] >CPD of magnitude:
+----------------+-------+
| magnitude(0.0) | 0.11  |
+----------------+-------+
| magnitude(1.0) | 0.214 |
+----------------+-------+
| magnitude(2.0) | 0.218 |
+----------------+-------+
| magnitude(3.0) | 0.223 |
+----------------+-------+
| magnitude(4.0) | 0.235 |
+----------------+-------+
[bnlearn] >CPD of depends_on_magnitude:
+---------------------------+----------------+-----+--------------------+----------------+
| magnitude                 | magnitude(0.0) | ... | magnitude(4.0)     | magnitude(4.0) |
+---------------------------+----------------+-----+--------------------+----------------+
| outcome                   | outcome(0.0)   | ... | outcome(3.0)       | outcome(4.0)   |
+---------------------------+----------------+-----+------------

# WeightTreeCreating

In [ ]:
import copy
def chain_from_bamt_bn_weights(
    bamt_bn_weights,
    outcome_column_name: str,
    top_n: int,
    influence_threshold: float = 0.1,
    depth=None,
    branch=None,
    chain=None
):
  if chain is None:
    chain = {}
  if depth is None:
    depth = 0
  if branch is None:
    branch = 0

  df_influence = pd.DataFrame(columns=['influence'])
  bamt_bn_weights_ = copy.copy(bamt_bn_weights)

  for components, influence in bamt_bn_weights.items():
    if outcome_column_name in components:
      del bamt_bn_weights_[components]
      pair_component = (set(components) - set([outcome_column_name])).pop()
      df_influence.loc[pair_component] = influence
  if df_influence.empty:
    return chain
  df_influence = df_influence.sort_values('influence', ascending=False)
  actors = None
  if influence_threshold is None:
    actors = list(df_influence.head(top_n).index)
  else:
    query = df_influence.query('influence >= @influence_threshold')
    if not query.empty:
      query = query.head(top_n)
      actors = list(query.index)
  chain[f'{depth}_{branch}::{outcome_column_name}'] = df_influence
  if actors is not None:
    for branch, actor in enumerate(actors):
      weights_to_go = copy.copy(bamt_bn_weights_)
      for other_actor in actors:
        for components in bamt_bn_weights_.keys():
          if (other_actor != actor) and (other_actor in components):
            if components in weights_to_go:
              del weights_to_go[components]

      chain = chain_from_bamt_bn_weights(
          weights_to_go,
          actor,
          top_n,
          influence_threshold,
          depth + 1,
          branch,
          chain
      )
  return chain



def interpret_chain(chain, influence_threshold: float = 0.1):
  interpreted_chain = {}
  for name, df_influence in chain.items():
    query = df_influence.query('influence >= @influence_threshold')
    if not query.empty:
      interpreted_chain[name] = []
      for actor, row in query.iterrows():
        interpreted_chain[name].append((actor, row['influence']))
  return interpreted_chain


"""
e.g.
interpret_chain(
  chain=chain_from_bamt_bn_weights(bamt_bn_weights, 'outcome', 3, 0.1),
  influence_threshold=0.1
)
"""

"\ne.g.\ninterpret_chain(\n  chain=chain_from_bamt_bn_weights(bamt_bn_weights, 'outcome', 3, 0.1),\n  influence_threshold=0.1\n)\n"

In [ ]:
interpret_chain(
  chain=chain_from_bamt_bn_weights(bn.weights, 'outcome', 1, 0.01),
  influence_threshold=0.01
)

{'0_0::outcome': [('depends_on_magnitude', 0.7590011098616545),
  ('magnitude', 0.7401386957995995),
  ('garbage_2', 0.17032773369992427)]}